# Color Detection with Zero-Shot Classification


# Objective & Scope

The primary goal of this assignment is to leverage a pre-trained **Transformer model (DistilBERT)** to perform a novel task: identifying and classifying colors within unstructured text using **Zero-Shot Classification**.

### Extended Analysis & Advanced Methodologies
While the assignment focuses on DistilBERT, this project extends the scope in two key areas:
1.  **Model Benchmarking:** Comparing the baseline against advanced architectures (**BART-Large** and **mDeBERTa-v3**) to evaluate performance differences.
2.  **Inference Optimization:** Implementing **Hypothesis Templating** to structure queries and **Input Boosting** (context augmentation) to improve the model's semantic understanding.

### Concepts Covered
* **Transformers & DistilBERT:** Understanding the purpose and utility of base models.
* **Tokenization & Embeddings:** Input preparation and representation.
* **Zero-Shot Classification:** Classifying text based on arbitrary labels provided at runtime.
* **Prompt Engineering:** Applying **Hypothesis Templating** and **Input Boosting** to refine model outputs.
* **Iteration & Data Structures:** Managing candidate labels (colors) and storing confidence scores.

In [1]:
!pip install transformer torch

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer


In [2]:
from transformers import pipeline
import transformers
import torch
import re

In [3]:
transformers.__version__

'4.57.1'

In [4]:
torch.__version__

'2.8.0+cu126'

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
device

device(type='cuda')

In [7]:
torch.cuda.current_device()

0

In [8]:
torch.cuda.get_device_name(torch.cuda.current_device())

'Tesla T4'

# Zero-Shot Baseline: DistilBERT

In [9]:
model= "typeform/distilbert-base-uncased-mnli"

In [10]:
classifier1 = pipeline("zero-shot-classification", model=model, device = device)

Device set to use cuda


In [11]:
text = "A red car drove past a house with a yellow door, while purple flowers bloomed near the brown wooden fence."
colors = ["red","orange","yellow", "green", "blue","purple", "black", "white", "brown"]
squares =  ["🟥", "🟧", "🟨", "🟩", "🟦", "🟪", "⬛", "⬜", "🟫"]
labels2squares = {label:square for label, square in zip(colors,squares)}

In [12]:
labels2squares

{'red': '🟥',
 'orange': '🟧',
 'yellow': '🟨',
 'green': '🟩',
 'blue': '🟦',
 'purple': '🟪',
 'black': '⬛',
 'white': '⬜',
 'brown': '🟫'}

In [13]:
outputs = classifier1(text, colors, multi_label = True)

In [14]:
outputs

{'sequence': 'A red car drove past a house with a yellow door, while purple flowers bloomed near the brown wooden fence.',
 'labels': ['yellow',
  'red',
  'brown',
  'purple',
  'green',
  'orange',
  'blue',
  'white',
  'black'],
 'scores': [0.9874433279037476,
  0.9762481451034546,
  0.4106666147708893,
  0.32382312417030334,
  0.0019113413291051984,
  0.0005098325782455504,
  0.00023173223598860204,
  0.00022911373525857925,
  5.40871660632547e-05]}

In [15]:
scores = outputs["scores"]
color_labels = outputs["labels"]

In [16]:
scores

[0.9874433279037476,
 0.9762481451034546,
 0.4106666147708893,
 0.32382312417030334,
 0.0019113413291051984,
 0.0005098325782455504,
 0.00023173223598860204,
 0.00022911373525857925,
 5.40871660632547e-05]

In [17]:
for score, color in zip(scores,color_labels):
  if score >= 0.3:
    print(f"{labels2squares[color]} : {score}")

🟨 : 0.9874433279037476
🟥 : 0.9762481451034546
🟫 : 0.4106666147708893
🟪 : 0.32382312417030334


In [53]:
def color_detection(text, conf_score = 0.1):
  model = "typeform/distilbert-base-uncased-mnli"

  classifier = pipeline("zero-shot-classification", model=model, device = device)

  colors = ["red","orange","yellow", "green", "blue","purple", "black", "white", "brown"]
  squares =  ["🟥", "🟧", "🟨", "🟩", "🟦", "🟪", "⬛", "⬜", "🟫"]
  labels2squares = {label:square for label, square in zip(colors,squares)}

  outputs = classifier1(text, colors, multi_label = True)

  scores = outputs["scores"]
  color_labels = outputs["labels"]


  for score, color in zip(scores,color_labels):
    if score >= conf_score:
      print(f"{labels2squares[color]} : {score}")

In [54]:
text = "A red car drove past a house with a yellow door, while purple flowers bloomed near the brown wooden fence."

In [55]:
color_detection(text)

Device set to use cuda


🟥 : 0.9917163252830505
🟪 : 0.8269215822219849
🟨 : 0.5092152953147888
🟫 : 0.2559010982513428


# Enhanced Zero-Shot Learning with DeBERTa

In [18]:
model1 = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"

In [19]:
classifier1 = pipeline("zero-shot-classification", model=model1, device = device)

Device set to use cuda


In [20]:
outputs = classifier1(text, colors, multi_label = True)

In [21]:
scores = outputs["scores"]
color_labels = outputs["labels"]

In [22]:
scores

[0.9917163252830505,
 0.8269215822219849,
 0.5092152953147888,
 0.2559010982513428,
 0.005968730431050062,
 0.005281334277242422,
 0.003876926377415657,
 0.0038022249937057495,
 0.002269468503072858]

In [23]:
for score, color in zip(scores,color_labels):
  if score >= 0.2:
    print(f"{labels2squares[color]} : {score}")

🟥 : 0.9917163252830505
🟪 : 0.8269215822219849
🟨 : 0.5092152953147888
🟫 : 0.2559010982513428


# Hypothesis Templating - Approach 1


In [24]:
hypotheses = [f"The text mentions something {color} in color." for color in colors]

In [25]:
outputs = classifier1(text, hypotheses, multi_label = True)

In [26]:
scores = outputs["scores"]
color_labels = outputs["labels"]

In [27]:
scores

[0.9847230315208435,
 0.9658029079437256,
 0.9490591287612915,
 0.7566273808479309,
 0.12885363399982452,
 0.03690806403756142,
 0.028463028371334076,
 0.023043232038617134,
 0.01548398844897747]

In [28]:
for score, color in zip(scores,color_labels):
  color = re.search(r"something (\w+) in color", color).group(1)
  if score >= 0.7:
    print(f"{labels2squares[color]} : {score}")

🟥 : 0.9847230315208435
🟪 : 0.9658029079437256
🟨 : 0.9490591287612915
🟫 : 0.7566273808479309


# NLI-Based Classification using Facebook/BART

In [29]:
model2 = "facebook/bart-large-mnli"

In [30]:
classifier2 = pipeline("zero-shot-classification", model = model2, device = device)

Device set to use cuda


In [31]:
text = "A red car drove past a house with a yellow door, while purple flowers bloomed near the brown wooden fence."

In [32]:
outputs = classifier2(text, hypotheses, multi_label = True)

In [33]:
scores = outputs["scores"]
color_labels = outputs["labels"]

In [34]:
scores

[0.9852646589279175,
 0.9766191244125366,
 0.9105637669563293,
 0.6993283629417419,
 0.0006850688951089978,
 0.0006749349995516241,
 0.0006748505984432995,
 0.0003156167222186923,
 0.00011459344386821613]

In [35]:
outputs

{'sequence': 'A red car drove past a house with a yellow door, while purple flowers bloomed near the brown wooden fence.',
 'labels': ['The text mentions something red in color.',
  'The text mentions something purple in color.',
  'The text mentions something yellow in color.',
  'The text mentions something brown in color.',
  'The text mentions something green in color.',
  'The text mentions something white in color.',
  'The text mentions something orange in color.',
  'The text mentions something blue in color.',
  'The text mentions something black in color.'],
 'scores': [0.9852646589279175,
  0.9766191244125366,
  0.9105637669563293,
  0.6993283629417419,
  0.0006850688951089978,
  0.0006749349995516241,
  0.0006748505984432995,
  0.0003156167222186923,
  0.00011459344386821613]}

In [36]:
for score, color in zip(scores,color_labels):
  color = re.search(r"something (\w+) in color", color).group(1)
  if score >= 0.5:
    print(f"{labels2squares[color]} : {score}")

🟥 : 0.9852646589279175
🟪 : 0.9766191244125366
🟨 : 0.9105637669563293
🟫 : 0.6993283629417419


# Batch Inference on Multiple Examples

In [37]:
sentences = [
    "The sky was a bright blue, and a few white clouds floated lazily above the green meadow.",
    "A red car drove past a house with a yellow door, while purple flowers bloomed near the brown wooden fence.",
    "In the evening, the sunset painted the horizon orange and red, fading slowly into black as night arrived."
]

In [38]:
confidence_list = []

for sentence in sentences:
  outputs = classifier1(sentence, hypotheses, multi_label = True)

  scores = outputs["scores"]
  color_labels = outputs["labels"]

  confidence = []

  for score, color in zip(scores,color_labels):
    color = re.search(r"something (\w+) in color", color).group(1)
    if score >= 0.3:

      confidence.append(f"{labels2squares[color]} : {score}")

  confidence_list.append(confidence)

In [39]:
for confidence in confidence_list:
  for conf in confidence:
    print(conf)
  print("\n")

🟦 : 0.9953994154930115
⬜ : 0.49188297986984253
🟩 : 0.3089715838432312


🟥 : 0.9847230315208435
🟪 : 0.9658029079437256
🟨 : 0.9490591287612915
🟫 : 0.7566273808479309


🟧 : 0.9847083687782288
⬛ : 0.963161289691925
🟥 : 0.9612074494361877




# Context/Input Boosting - Approach 2

In [40]:
text = "A red car drove past a house with a yellow door, while purple flowers bloomed near the brown wooden fence."

In [41]:
booster = "This sentence might mention a color"

In [42]:
boosted_sentence = text + booster

In [43]:
outputs = classifier1(boosted_sentence, hypotheses, multi_label=True)
scores = outputs["scores"]
labels = outputs["labels"]

for score, label in zip(scores, labels):
    color = re.search(r"something (\w+) in color", label).group(1)
    if score >0.5:
        print(f"{labels2squares[color]} : {score}")

🟪 : 0.9955748319625854
🟥 : 0.9905874133110046
🟨 : 0.9510514140129089
🟫 : 0.9302684664726257


# Testing on Custom User Input

In [44]:
sentence = str(input("Write a sentence: "))
print(f"The sentence: {sentence}")

Write a sentence: The red and orange leaves fell softly onto the green grass under a deep blue sky, while a black bird perched silently on a branch.
The sentence: The red and orange leaves fell softly onto the green grass under a deep blue sky, while a black bird perched silently on a branch.


In [45]:
boosted_sentence = sentence + booster

In [46]:
print(boosted_sentence)

The red and orange leaves fell softly onto the green grass under a deep blue sky, while a black bird perched silently on a branch.This sentence might mention a color


In [47]:
colors = ["red","orange","yellow", "green", "blue","purple", "black", "white", "brown"]
squares =  ["🟥", "🟧", "🟨", "🟩", "🟦", "🟪", "⬛", "⬜", "🟫"]
labels2squares = {label:square for label, square in zip(colors,squares)}
hypotheses = [f"The text mentions something {color} in color." for color in colors]

In [48]:
outputs = classifier1(boosted_sentence, hypotheses, multi_label=True)

In [49]:
outputs = classifier1(boosted_sentence, hypotheses, multi_label=True)
scores = outputs["scores"]
labels = outputs["labels"]

for score, label in zip(scores, labels):
    color = re.search(r"something (\w+) in color", label).group(1)
    if score >0.5:
        print(f"{labels2squares[color]} : {score}")

🟥 : 0.996827244758606
🟦 : 0.9960469603538513
🟧 : 0.9958256483078003
⬛ : 0.9510968327522278
🟩 : 0.9360515475273132
